In [1]:
%pylab
from glob import glob
from natsort import natsorted
import h5py
from scipy.stats import binned_statistic

/Users/mgrudic/intelpython3/lib/python3.7/site-packages/IPython/core/pylabtools.py:276: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  import matplotlib


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import palettable.colorbrewer.qualitative
from matplotlib.ticker import ScalarFormatter, NullFormatter,FormatStrFormatter

from palettable.colorbrewer.qualitative import Dark2_7
#from palettable.cubehelix import Cubehelix
#palette = Cubehelix.make(n=10)

In [6]:
plt.clf()
fig, ax = plt.subplots(2,4,figsize=(8,4))
fig.delaxes(ax[0,3])
dirs = natsorted(glob("sinkdata/jettests2/tests/M2e3*/output*"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Dark2_7.mpl_colors)

M0 = 2e3

labeldict = {"T10": r"$t_{\rm acc} \times 10$",
             "THETA10": r"$\theta_{\rm 0} = 0.1$",
             "THETA100": r"$\theta_{\rm 0} = 1$",
             "NEW": r"Reference",
             "ISOTROPIC": r"Isotropic Jets",
             "HEATING": r"Protostellar Heating",
             "HEATING10": r"Protostellar Heating $\times10$",
             "f0.5": r"No jets; $f_{\rm acc}=0.5$",
#             "NEW1e-05": r"$\Delta m_{\rm w} = 0.01\Delta m$",
             "NEW0.001": r"$\Delta m_{\rm w} = \Delta m$",
             "m0.1": r"No jets if $<0.1M_\odot$",
              "m1": r"No jets if $<M_\odot$",
             "JRETURN": r"Ang. mom. return",
             "NOMHD": r"No MHD",
             "NOFB": r"No jets"
#             "ISOT": r"Isothermal"
#             "WIND": r"+Stellar Winds",
#            "THINONLY":r"no opacity limit"
#             "THETA100": 
            
            }

plt.rcParams.update({'axes.labelsize': 8})
#plt.rcParams.update({'axes.ticklabelsize': 8})

plt.rc('xtick',labelsize=6)
plt.rc('ytick',labelsize=6)

i = 0
for d in dirs:
    print(d)
    name = d.split("output_jets_")[1].split("_fw")[0]
#    print(name)
    if not "fw0.3_fk0.3" in d: continue
    if not "m0.0001" in d: 
        name += d.split("_m")[1].replace("/","")
    print(name)
    if not name in labeldict.keys(): continue
#    if not "fw0.3_fk0.3" in d: continue
#    if not ("THETA" in d or "ISOTROPIC" in d): continue
    if glob(d+"/snapshot*stars.hdf5"):
        files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    else:
        continue
        #files = natsorted(glob(d+"/output_old/snapshot*stars.hdf5"))

    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    machs = []
    mpeak = []
    
#    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
#    dm = dm.replace("-0","{-}")
    for f in files:
        #print(f)
        with h5py.File(f,'r') as F:
            if not "PartType5" in F.keys(): continue
            mstar = np.array(F["PartType5"]["Masses"])
#            if not (mstar>0.08).sum(): continue
#            mstar = mstar[mstar>0.08]
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())
            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            machs.append(np.array(F["Mach"]))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    ls = ('solid' if i < 7 else 'dashed')
    color = ('black' if name=="NEW" else None)
    z = (1000 if name=="NEW" else 1)
    ax[0,0].plot(times-times[mtot>0][0], mtot/M0, label=labeldict[name], color=color,ls=ls,zorder=z,lw=1)#,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
#    ax[0,0].plot([-1,-1],label="Kroupa IMF")
    ax[0,1].plot(mtot, Nsink,lw=1,ls=ls,zorder=z,color=color)
    ax[0,2].plot(mtot, mquantiles[:,2],lw=1,ls=ls,zorder=z,color=color)
#    ax[0,2].plot([0,1000],[1.8,1.8],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(mtot, quantiles[:,2],lw=1,ls=ls,zorder=z,color=color)
#    ax[1,0].plot([0,1000],[0.1,0.1],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(mtot, means,lw=1,ls=ls,zorder=z,color=color)
#    ax[1,1].plot([0,1000],[0.398,0.398],ls='dashed',color='black',zorder=-100)
    ax[1,3].plot(mtot, quantiles[:,-1],lw=1,ls=ls,zorder=z,color=color)
    ax[1,0].plot(times-times[mtot>0][0], machs,zorder=z,lw=1,ls=ls,color=color)
    i += 1
ax[0,0].set(yscale='log',xlim=[1e-1,6],ylim=[1e-3,1], ylabel="SFE",xlabel="Time (Myr)",xscale='log')
#ax[0,0].set_xlabel("Time(Myr)", fontsize=8)
ax[0,1].set(yscale='log',xscale='log',ylim=[0.5,500], xlim=[1,250],ylabel="$N_\mathrm{*}$",xlabel=r"$M_{\rm \star,tot}$ $(M_\odot)$")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.3,20], xlim=[1,250], ylabel="$M_\mathrm{50}\,(M_\odot)$",xlabel=r"$M_{\rm \star,tot}$ $(M_\odot)$")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.03,10], xlim=[1,250], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel=r"$M_{\rm \star,tot}$ $(M_\odot)$")
ax[1,2].set(yscale='log',xscale='log',ylim=[0.1,10], xlim=[1,250],ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel=r"$M_{\rm \star,tot}$ $(M_\odot)$")
ax[1,3].set(yscale='log',xscale='log',ylim=[1,100], xlim=[1,250], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel=r"$M_{\rm \star,tot}$ $(M_\odot)$")
ax[1,0].set(xscale='log',xlim=[1e-1,6],xlabel='Time (Myr)',ylabel="$\mathcal{M}$")
#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


f = FormatStrFormatter("%g")
f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
#ax[1,0].xaxis.set_major_formatter(f)
#ax[,0].xaxis.set_minor_formatter(f)
for a in ax.flatten()[1:]:
    a.yaxis.set_major_formatter(f)
    a.xaxis.set_major_formatter(f)


ax[0,0].legend(labelspacing=0,fontsize=7,bbox_to_anchor=(5.3,1),bbox_transform=ax[0,0].transAxes, ncol=1)

fig.subplots_adjust(hspace=0.37,wspace=0.35)
plt.savefig("M2e3_Jets_Numerics_vs_Mstar.pdf",bbox_inches='tight')

sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_ADAPTIVE_0.25_fw0.3_fk0.3_m0.0001
ADAPTIVE_0.25
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_ADAPTIVE_fw0.3_fk0.3_m0.0001
ADAPTIVE
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_BENCHMARK_fw0.3_fk0.3_m0.0001
BENCHMARK
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_BRETURN_UNWEIGHTED_fw0.3_fk0.3_m0.0001
BRETURN_UNWEIGHTED
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_BRETURN_fw0.3_fk0.3_m0.0001
BRETURN
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_HEATING0.1_fw0.3_fk0.3_m0.0001
HEATING0.1
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_HEATING10_fw0.3_fk0.3_m0.0001
HEATING10


/Users/mgrudic/intelpython3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in log10


sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_HEATING_fw0.3_fk0.3_m0.0001
HEATING
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_HEATING_noMS_fw0.3_fk0.3_m0.0001
HEATING_noMS
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_ISOTROPIC_fw0.3_fk0.3_m0.0001
ISOTROPIC
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_ISOT_fw0.3_fk0.3_m0.0001
ISOT
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_JRETURN_fw0.3_fk0.3_m0.0001
JRETURN
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_MOLECULAR_fw0.3_fk0.3_m0.0001
MOLECULAR
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_NEW_fw0.1_fk0.3_m0.001
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_NEW_fw0.1_fk0.3_m0.0001
sinkdata/jettests2/tests/M2e3_R3_S0_T1_B0.01_Res126_n2_sol0.5_42/output_jets_NEW_fw0.1_fk0.3_m1e-05
sinkdata/

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,3,figsize=(8,4),sharex=True)
dirs = natsorted(glob("sinkdata/jettests2/tests/convergence/M2e3*/output"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Haline_10_r.mpl_colors)

M0 = 2e3
for d in dirs:
    files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    mpeak = []
    
    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
    dm = dm.replace("-0","{-}")
    for f in files:
        print(f)
        with h5py.File(f,'r') as F:
            if not "PartType5" in F.keys(): continue                
            mstar = np.array(F["PartType5"]["BH_Mass"])
            mstar = mstar[np.array(F["PartType5"]["BH_Mass"])/np.array(F["PartType5"]["BH_Mdot"])*1e9 > 1e8]
            if len(mstar)<2: continue
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())
            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    ax[0,0].plot(times, mtot/M0,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    ax[0,1].plot(times, Nsink,lw=1)
    ax[0,2].plot(times, mquantiles[:,2],lw=1)
    ax[0,2].plot([0,10],[1.8,1.8],ls='dashed',color='black',zorder=-100)
    ax[1,0].plot(times, quantiles[:,2],lw=1)
    ax[1,0].plot([0,10],[0.1,0.1],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(times, means,lw=1)
    ax[1,1].plot([0,10],[0.398,0.398],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(times, quantiles[:,-1],lw=1)
ax[0,0].set(yscale='log',xscale='log',xlim=[1,9],ylim=[3e-3,0.3], ylabel="SFE")
ax[0,1].set(yscale='log',xscale='log',ylim=[0.5,500], ylabel="$N_\mathrm{*}$")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.3,30], ylabel="$M_\mathrm{50}\,(M_\odot)$")
ax[1,0].set(yscale='log',xscale='log',ylim=[0.03,3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.1,10], ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,2].set(yscale='log',xscale='log',ylim=[1,110], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel="Time (Myr)")

#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


f = FormatStrFormatter("%g")
f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
ax[1,0].xaxis.set_major_formatter(f)
ax[1,0].xaxis.set_minor_formatter(f)
for a in ax.flatten():
    a.yaxis.set_major_formatter(f)



ax[0,0].legend(labelspacing=0,fontsize=8,bbox_to_anchor=(3.6,1.22),bbox_transform=ax[0,0].transAxes, ncol=5)

fig.subplots_adjust(hspace=0,wspace=0.3)
plt.savefig("M2e3_Jets_MassResolution_NonAccreting.pdf",bbox_inches='tight')
    

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,3,figsize=(8,4),sharex=True)
dirs = natsorted(glob("sinkdata/jettests2/tests/convergence_heating/M2e3*/output"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Haline_10_r.mpl_colors)

M0 = 2e3
for d in dirs:
    files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    mpeak = []
    
    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
    dm = dm.replace("-0","{-}")
    for f in files:
        with h5py.File(f,'r') as F:
            if not "PartType5" in F.keys(): continue
            mstar = np.array(F["PartType5"]["BH_Mass"])
#            if not (mstar>0.08).sum(): continue
#            mstar = mstar[mstar>0.08]
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())
            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    if len(mquantiles) == 0: continue
    ax[0,0].plot(times, mtot/M0,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    ax[0,1].plot(times, Nsink,lw=1)
    ax[0,2].plot(times, mquantiles[:,2],lw=1)
    ax[0,2].plot([0,10],[1.8,1.8],ls='dashed',color='black',zorder=-100)
    ax[1,0].plot(times, quantiles[:,2],lw=1)
    ax[1,0].plot([0,10],[0.1,0.1],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(times, means,lw=1)
    ax[1,1].plot([0,10],[0.398,0.398],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(times, quantiles[:,-1],lw=1)
ax[0,0].set(yscale='log',xscale='log',xlim=[1,9],ylim=[3e-3,0.3], ylabel="SFE")
ax[0,1].set(yscale='log',xscale='log',ylim=[0.5,500], ylabel="$N_\mathrm{*}$")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.3,30], ylabel="$M_\mathrm{50}\,(M_\odot)$")
ax[1,0].set(yscale='log',xscale='log',ylim=[0.03,3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.1,10], ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,2].set(yscale='log',xscale='log',ylim=[1,110], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel="Time (Myr)")

#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


f = FormatStrFormatter("%g")
f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
ax[1,0].xaxis.set_major_formatter(f)
ax[1,0].xaxis.set_minor_formatter(f)
for a in ax.flatten():
    a.yaxis.set_major_formatter(f)



ax[0,0].legend(labelspacing=0,fontsize=8,bbox_to_anchor=(3.6,1.22),bbox_transform=ax[0,0].transAxes, ncol=5)

fig.subplots_adjust(hspace=0,wspace=0.3)
plt.savefig("M2e3_Jets_Heating_MassResolution.pdf",bbox_inches='tight')
    

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,3,figsize=(8,4),sharex=True)
dirs = natsorted(glob("sinkdata/jettests2/tests/convergence_heating/M2e3*/output"))
q = np.array([0,14,50,86,100])

for a in ax.flatten():
    a.set_prop_cycle('color', Haline_10_r.mpl_colors)

M0 = 2e3
for d in dirs:
    files = natsorted(glob(d+"/snapshot*stars.hdf5"))
    quantiles = []
    means = []
    mquantiles = []
    mtot = []
    times = []
    Nsink = []
    mpeak = []
    
    dm = "%1.0e"%(M0 / float(d.split("Res")[1].split("_")[0])**3)
    dm = dm.replace("-0","{-}")
    for f in files:
        with h5py.File(f,'r') as F:
            if not "PartType5" in F.keys(): continue
            mstar = np.array(F["PartType5"]["BH_Mass"])
            if not (mstar>0.08).sum(): continue
            mstar = mstar[mstar>0.08]
            means.append(mstar.mean())
            quantiles.append(np.percentile(mstar, q))
            mquantiles.append(np.interp(np.log10(mstar.sum()*q/100), np.log10(np.sort(mstar).cumsum()), np.sort(mstar)))
            mtot.append(mstar.sum())
            times.append(F["Header"].attrs["Time"]*979)
            Nsink.append(len(mstar))
            #print(F["Header"].attrs.keys())
            npeak = np.array([np.sum(np.abs(np.log10(mm/mstar))<0.1) for mm in mstar])
            mpeak.append(mstar[npeak.argmax()])
    quantiles = np.array(quantiles)
    mquantiles = np.array(mquantiles)
    means = np.array(means)
    mtot = np.array(mtot)
    times = np.array(times)
    Nsink = np.array(Nsink)
    mpeak = np.array(mpeak)
    ax[0,0].plot(times, mtot/M0,label="$\Delta m{=}%sM_\odot$"%dm,lw=1)
    ax[0,1].plot(times, Nsink,lw=1)
    ax[0,2].plot(times, mquantiles[:,2],lw=1)
    ax[0,2].plot([0,10],[2,2],ls='dashed',color='black',zorder=-100)
    ax[1,0].plot(times, quantiles[:,2],lw=1)
    ax[1,0].plot([0,10],[0.23,0.23],ls='dashed',color='black',zorder=-100)
    ax[1,1].plot(times, means,lw=1)
    ax[1,1].plot([0,10],[0.6,0.6],ls='dashed',color='black',zorder=-100)
    ax[1,2].plot(times, quantiles[:,-1],lw=1)
ax[0,0].set(yscale='log',xscale='log',xlim=[1,9],ylim=[3e-3,0.3], ylabel="SFE")
ax[0,1].set(yscale='log',xscale='log',ylim=[0.5,500], ylabel="$N_\mathrm{*}$")
ax[0,2].set(yscale='log',xscale='log',ylim=[0.3,30], ylabel="$M_\mathrm{50}\,(M_\odot)$")
ax[1,0].set(yscale='log',xscale='log',ylim=[0.03,3], ylabel="$M_\mathrm{med}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,1].set(yscale='log',xscale='log',ylim=[0.1,10], ylabel="$M_\mathrm{mean}\,(M_\odot)$",xlabel="Time (Myr)")
ax[1,2].set(yscale='log',xscale='log',ylim=[1,110], ylabel="$M_\mathrm{max}\,(M_\odot)$",xlabel="Time (Myr)")

#ax[1,2].yaxis.set_label_position("right")
#ax[0,2].yaxis.set_label_position("right")
#ax[1,2].yaxis.tick_right()#yaxis.("right")
#ax[0,2].yaxis.tick_right()#yaxis.set_ticklabel_position("right")


f = FormatStrFormatter("%g")
f2=ScalarFormatter()
#ax[1,0].set_xticks([1,10])
ax[1,0].xaxis.set_major_formatter(f)
ax[1,0].xaxis.set_minor_formatter(f)
for a in ax.flatten():
    a.yaxis.set_major_formatter(f)



ax[0,0].legend(labelspacing=0,fontsize=8,bbox_to_anchor=(3.6,1.22),bbox_transform=ax[0,0].transAxes, ncol=5)

fig.subplots_adjust(hspace=0,wspace=0.3)
plt.savefig("M2e3_Jets_MassResolution_Heating_NoBrownDwarfs.pdf",bbox_inches='tight')